In [55]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.impute import KNNImputer

In [60]:
file_path = r'.\raw_data\boy or girl 2024 test no ans_missingValue.csv'
data = pd.read_csv(file_path)
data

,id,gender,star_sign,phone_os,height,weight,sleepiness,iq,fb_friends,yt,self_intro
0,1,0,天蠍座,Apple,NaN,100,1,87,87,87,GOod
1,2,0,金牛座,Apple,175,80,3,130,2.0E+03,30,Easygoing
2,3,0,雙子座,Apple,155,45,3,150,400,9,I LOVE INTEL
3,4,0,處女座,Apple,173,85,4,100,2.0E+03,15,"I'm a hard-work man, just do my best to finish..."
4,5,0,射手座,Android,164,57,4,130,505,2,I'm smart
...,...,...,...,...,...,...,...,...,...,...,...
390,391,0,處女座,Android,160,48,3,75,98,2,Starting by Starting
391,392,0,NaN,Apple,170,NaN,NaN,105,510,NaN,A little bit smart
392,393,0,金牛座,Apple,160,45,4,100,600,2.0E+03,Hi
393,394,0,巨蟹座,NaN,180,NaN,NaN,200,NaN,60,"I'm not beautiful, but smart 😀"


In [61]:
# 星座的映射字典
star_sign_mapping = {
    '水瓶座': 0,
    '雙魚座': 1,
    '牡羊座': 2,
    '金牛座': 3,
    '雙子座': 4,
    '巨蟹座': 5,
    '獅子座': 6,
    '處女座': 7,
    '天秤座': 8,
    '天蠍座': 9,
    '射手座': 10,
    '摩羯座': 11
}

# 將星座欄位轉換為數字
data['star_sign'] = data['star_sign'].map(star_sign_mapping)

# 手機操作系統的映射字典
phone_os_mapping = {
    'Apple': 0,
    'Android': 1,
    'Windows phone': 2  
}

# 將手機操作系統欄位轉換為數字
data['phone_os'] = data['phone_os'].map(phone_os_mapping)

# 輸出處理後的測試資料集
data

,id,gender,star_sign,phone_os,height,weight,sleepiness,iq,fb_friends,yt,self_intro
0,1,0,9,0,NaN,100,1,87,87,87,GOod
1,2,0,3,0,175,80,3,130,2.0E+03,30,Easygoing
2,3,0,4,0,155,45,3,150,400,9,I LOVE INTEL
3,4,0,7,0,173,85,4,100,2.0E+03,15,"I'm a hard-work man, just do my best to finish..."
4,5,0,10,1,164,57,4,130,505,2,I'm smart
...,...,...,...,...,...,...,...,...,...,...,...
390,391,0,7,1,160,48,3,75,98,2,Starting by Starting
391,392,0,NaN,0,170,NaN,NaN,105,510,NaN,A little bit smart
392,393,0,3,0,160,45,4,100,600,2.0E+03,Hi
393,394,0,5,NaN,180,NaN,NaN,200,NaN,60,"I'm not beautiful, but smart 😀"


In [62]:
def calculate_bmi(height, weight):
    return weight / ((height/100) ** 2)

In [44]:
# data['yt'] = pd.to_numeric(data['yt'], errors='coerce')

In [63]:
# 将'#NUM!'替换为NaN
data.replace('#NUM!', np.nan, inplace=True)

# 設置 pandas 顯示格式
pd.set_option('display.float_format', lambda x: '%.1E' % x if abs(x) > 1000 else '%.0f' % x)

# 欄位不包含 'id' 和 'self_intro'
columns_to_check = [col for col in data.columns if col not in ['id', 'self_intro']]

# 初始化計數器
total_outliers = 0
outliers_gender_1 = 0
outliers_gender_2 = 0

# 找出有outlier的資料
outliers = []
for index, row in data.iterrows():
    is_outlier = False
    for col in columns_to_check:
        # 如果該欄位是負值，則視為outlier
        if row[col] < 0:
            is_outlier = True
            break
        # 如果該欄位的數值過大，則視為outlier
        if col in ['height'] and (row[col] > 220 or row[col] < 140):  # 可以自行調整數值範圍
            is_outlier = True
            break
        
        if col in ['weight'] and (row[col] > 150 or row[col] < 30):  # 可以自行調整數值範圍
            is_outlier = True
            break

        if col in ['fb_friends'] and row[col] > 10000:  # 可以自行調整數值範圍
            is_outlier = True
            break

        if col in ['yt'] and row[col] > 5000:  # 可以自行調整數值範圍
            is_outlier = True
            break

        if col in ['bmi'] and (row[col] > 40 or row[col]<10):
            is_outlier = True
            break

        # 計算z-score
        z_score = zscore([row[col]])[0]
        # 如果z-score的絕對值大於3，則視為outlier
        if abs(z_score) > 3:
            is_outlier = True
            break

    if is_outlier:
        outliers.append(index)

        total_outliers += 1
        if row['gender'] == 1:
            outliers_gender_1 += 1
        elif row['gender'] == 2:
            outliers_gender_2 += 1

# 列印出有outlier的資料
if outliers:
    print("有outlier的資料:")
    print(data.loc[outliers])
else:
    print("沒有outlier的資料")

# 列印出outlier的總筆數和gender為1和2的outlier筆數
print("Outlier的總筆數:", total_outliers)
print("Gender為1的Outlier筆數:", outliers_gender_1)
print("Gender為2的Outlier筆數:", outliers_gender_2)

outliers_csv = pd.DataFrame(data.loc[outliers])

# 将 outlier 替换为 NaN
for index in outliers:
    row = data.loc[index]
    for col in columns_to_check:
        try:
            if row[col] < 0 or (col == 'height' and (row[col] < 140 or row[col] > 250)) or \
               (col == 'weight' and (row[col] < 30 or row[col] > 200)) or \
               (col == 'fb_friends' and row[col] > 10000) or \
               (col == 'yt' and row[col] > 5000) or \
               (col == 'bmi' and (row[col] < 15 or row[col] > 35)):
                data.at[index, col] = np.nan
        except Exception as e:
            print(f"Error occurred while processing column: {col}")
            print(f"Error message: {str(e)}")

# print(data)

# 分別處理男生和女生的缺失值並使用 KNN 補值
def handle_outliers_and_impute(data, gender):
    gender_indices = data[data['gender'] == gender].index.tolist()
    numeric_cols = data.select_dtypes(include=['number']).columns.tolist()
    cleaned_data = data[numeric_cols]
    imputer = KNNImputer(n_neighbors=5)
    imputed_data = cleaned_data.copy()
    imputed_data.loc[gender_indices] = imputer.fit_transform(imputed_data.loc[gender_indices])
    return imputed_data

# 處理男生的缺失值並用 KNN 補值
if outliers_gender_1 > 0:
    gender_1_outliers_indices = data[(data['gender'] == 1) & data.index.isin(outliers)].index
    data.loc[gender_1_outliers_indices] = handle_outliers_and_impute(data.loc[gender_1_outliers_indices], 1)

# 處理女生的缺失值並用 KNN 補值
if outliers_gender_2 > 0:
    gender_2_outliers_indices = data[(data['gender'] == 2) & data.index.isin(outliers)].index
    data.loc[gender_2_outliers_indices] = handle_outliers_and_impute(data.loc[gender_2_outliers_indices], 2)

# 顯示處理後的資料
print(data.info())
print(data)

有outlier的資料:
      id  gender  star_sign  phone_os  height  weight  sleepiness  iq  \
15    16       0        NaN         1     188      20         NaN  88   
51    52       0          4         2      88      88           1  87   
61    62       0          9         1     170       1           4  50   
64    65       0          7         0     166      90           1 170   
88    89       0          8         1 2.0E+05 4.5E+05           2 180   
92    93       0          5         0 1.8E+21 1.8E+30           1 200   
121  122       0          6         0     160      50           4 180   
126  127       0          2         1      52      29           1  51   
128  129       0         11         0     174     888           1  50   
140  141       0          3         1     166      66           5 100   
157  158       0          4         0     171      66           5 199   
161  162       0          2         0     172      56           2 200   
181  182       0          5         1 

In [64]:
data

,id,gender,star_sign,phone_os,height,weight,sleepiness,iq,fb_friends,yt,self_intro
0,1,0,9,0,NaN,100,1,87,87,87,GOod
1,2,0,3,0,175,80,3,130,2.0E+03,30,Easygoing
2,3,0,4,0,155,45,3,150,400,9,I LOVE INTEL
3,4,0,7,0,173,85,4,100,2.0E+03,15,"I'm a hard-work man, just do my best to finish..."
4,5,0,10,1,164,57,4,130,505,2,I'm smart
...,...,...,...,...,...,...,...,...,...,...,...
390,391,0,7,1,160,48,3,75,98,2,Starting by Starting
391,392,0,NaN,0,170,NaN,NaN,105,510,NaN,A little bit smart
392,393,0,3,0,160,45,4,100,600,2.0E+03,Hi
393,394,0,5,NaN,180,NaN,NaN,200,NaN,60,"I'm not beautiful, but smart 😀"
